In [31]:
import math
import numpy as np
import os
from collections import Counter
import pandas as pd
import random
import shutil
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm
import sys


def Regression(data,norm=True, minsamples=120, linear=True):
    """
    Perform non linear regression, return beta, p_beta, alpha, p_alpha, se_alpha, se_beta
    data.columns has to have [allele1='x1', allele2='x2', gene_expression='expr'] not normalized
    """
    if data.shape[0] <= minsamples: 
        return None, None, None, None, None, None
    m = np.mean(list(locus_str["x1"].astype(int))+list(locus_str["x2"].astype(int))) 
    sd= math.sqrt(np.var(locus_str["x1"].astype(int) + locus_str["x2"].astype(int) ))
    if norm:
        data['x1'] = ZNorm(data['x1'].astype(int), sd, m)
        data['x2'] = ZNorm(data['x2'].astype(int), sd, m)
        E=data['expr'].astype(float)
        data['expr'] = ZNorm(E, np.mean(E), math.sqrt(np.var(E)))
        if data['x1'].isnull().all()  or data['x2'].isnull().all() or data['expr'].isnull().all(): 
            return None, None, None, None, None, None  
    data['X']=data['x1']+data['x2']
    data['X2']=data['x1']**2 + data['x2']**2
    model = sm.ols(formula = 'expr ~ X + X2', data = data).fit()
    #model = sm.OLS(Locus_detail[['expr']],Locus_detail[['X','X2']] , missing='drop').fit()
    #print(data.head(3), '\n..(1)..\n',data.head(3))
    mod_ols = sm.ols(formula = 'expr ~ X + X2', data = data).fit()
    alpha = list(mod_ols.params)[1]
    beta = list(mod_ols.params)[2]
    alpha_p = list(mod_ols.pvalues)[1]
    beta_p  = list(mod_ols.pvalues)[2]
    alpha_se=list(mod_ols.bse)[1]
    beta_se=list(mod_ols.bse)[2]
    quad_rsq = mod_ols.rsquared
    if linear: 
        #print(data.head(3), '\n..(2)..\n',data.head(3))
        X=data['X'].astype(int).values
        mod_ols1 = sm.OLS(data['expr'], X, missing='drop').fit()
        pval = mod_ols1.pvalues[0]
        slope = mod_ols1.params[0]
        err = mod_ols1.bse[0]
        lin_rsq = mod_ols1.rsquared
        anova_results = anova_lm(mod_ols, mod_ols1)
        anova_pval = anova_results["Pr(>F)"].values[1]
        delta_aic = mod_ols.aic - mod_ols1.aic
        
        return alpha, alpha_se, alpha_p, beta, beta_se, beta_p, slope, err, pval,quad_rsq, lin_rsq, delta_aic, anova_pval
    else:
        return alpha, alpha_se, alpha_p, beta, beta_se, beta_p
    
    


def ZNorm(vals,m,sd):
    if m is None:
        m = np.mean(vals)
        sd = math.sqrt(np.var(vals))
    if sd == 0: return None
    return [(item-m)/sd for item in vals]

def LinearRegression(data, Y, norm=False, minsamples=0, alleles=False):
    """
    Perform linear regression, return beta, beta_se, p
    """

    if norm:
        if alleles:
            data['x1']=data['x1'].astype(float) ; data['x2']=data['x2'].astype(float)
            data['x1+x2'] = data[['x1', 'x2']].sum(axis=1)
            X = ZNorm(data['x1+x2'], None, None)
        else:
            X = ZNorm(data) 
            Y = ZNorm(Y, None, None)
        if X is None or Y is None: return None, None, None
        if np.var(X)==0: return None, None, None
        if len(X) <= minsamples: return None, None, None
    else:
        X = data
    mod_ols = sm.OLS(Y, X, missing='drop')
    res_ols = mod_ols.fit()
    #print(res_ols.pvalues)
    pval = res_ols.pvalues[0]
    #print 'P-value: ', pval 
    slope = res_ols.params[0]
    err = res_ols.bse[0]
    return res_ols, slope, err, pval

def QuadraticRegression(data,norm=True, minsamples=120):
    """
    Perform non linear regression, return beta, p_beta, alpha, p_alpha, se_alpha, se_beta
    data.columns has to have [allele1='x1', allele2='x2', gene_expression='expr'] not normalized
    """
    if data.shape[0] <= minsamples: 
        return None, None, None, None, None, None, None
    m = np.mean(list(locus_str["x1"].astype(int))+list(locus_str["x2"].astype(int))) 
    sd= math.sqrt(np.var(locus_str["x1"].astype(int) + locus_str["x2"].astype(int) ))
    if norm:
        #data['x1'] = ZNorm(data['x1'].astype(int), sd, m)  
        #data['x2'] = ZNorm(data['x2'].astype(int), sd, m)
        #data['expr'] = ZNorm(data['expr'].astype(float), None, None)
        data.loc[:,'x1'] =ZNorm(data['x1'].astype(int), sd, m)
        data.loc[:,'x2'] =ZNorm(data['x2'].astype(int), sd, m)       
        data.loc[:,'expr'] =ZNorm(data['expr'].astype(int), sd, m)
        
        if data['x1'].isnull().all()  or data['x2'].isnull().all() or data['expr'].isnull().all(): 
            return None, None, None, None, None, None, None  
        
    data['X']=data['x1']+data['x2']
    data['X2']=data['x1']**2 + data['x2']**2
    #mod_ols = sm.OLS('expr ,X ,X2', data = data).fit()
    mod_ols = smf.ols(formula = 'expr ~ X + X2', data = data).fit()

    alpha = list(mod_ols.params)[1]
    beta = list(mod_ols.params)[2]
    alpha_p = list(mod_ols.pvalues)[1]
    beta_p  = list(mod_ols.pvalues)[2]
    alpha_se=list(mod_ols.bse)[1]
    beta_se=list(mod_ols.bse)[2]
    return mod_ols, alpha, alpha_se, alpha_p, beta, beta_se, beta_p

def Modelcompare(model1, model2):
    """
    Performs ANOVA test to compares 2 models 
    Outputs rsquares , delta AIC (aic_model1- aic_model2) and pvalue
    """
    rsq1 = model1.rsquared
    rsq2 = model2.rsquared
    anova_output = sm.stats.anova_lm(model1 , model2)
    #print(anova_output, anova_output["Pr(>F)"])
    anova_pval = anova_output["Pr(>F)"].values[1]
    delta_aic = model1.aic - model2.aic  
    return rsq1, rsq2, delta_aic, anova_pval

STRGTFILE= '/storage/szfeupe/Runs/650GTEx_estr/Genotypes/Allele_Gentypes.table'
EXPRESSION='/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/Adipose-Subcutaneous/Corr_Expr.csv'
EXPRANNOT='/storage/resources/dbase/human/hg19/gencode_gene_annotations_hg19.csv'
distfromgene = 100000
CHROM='chr2'
NORM=True

In [32]:
# Load STR genotypes
print("Load STRs")
strgt = pd.read_csv(STRGTFILE, sep="\t", low_memory=False)
strgt = strgt[strgt["chrom"] == CHROM]
print("Load expression")
expr = pd.read_csv(EXPRESSION)
print("Load annotation")
expr_annot = pd.read_csv(EXPRANNOT)
expr_annot.index = expr_annot["probe.id"].values
expr_annot = expr_annot[expr_annot["gene.chr"] == CHROM]
expr_annot = expr_annot.loc[[item for item in expr.columns if item in expr_annot.index],:]
# Restrict to STR samples
str_samples = list(set(strgt.columns[2:].values))
samples_to_remove = []
for item in str_samples:
    if item not in expr.index: samples_to_remove.append(item) #str_samples.remove(item)
for item in samples_to_remove: str_samples.remove(item)
expr = expr.loc[str_samples,:]
strgt = strgt[['chrom','start']+str_samples]
print("There are %s samples"%str(strgt.shape))


Load STRs
Load expression
Load annotation
There are (3445, 272) samples


In [36]:
NORM=True; ALLELE=True
MINSAMPLES=150
for i in range(150,expr_annot.shape[0],1):
    gene = expr_annot.index.values[i]
    print(" Getting data for %s"%gene)
    start = expr_annot["gene.start"].values[i]
    end = expr_annot["gene.stop"].values[i]
    cis_strs = strgt[(strgt["start"] >= (start-distfromgene)) & (strgt["start"] <= (end+distfromgene))]
    print(gene, '\t',start,'\t',end,'\t',"%s STRs tested \n"%str(cis_strs.shape[0]))
    y = pd.DataFrame({"expr":list(expr.loc[:, gene])})
    y.index = str_samples
    
    for j in range(cis_strs.shape[0]):
        locus_str = cis_strs.iloc[[j],:][str_samples].transpose()
        locus_str.index = str_samples
        locus_str.columns = ["STR_%s"%(cis_strs["start"].values[j])]
        test_str=locus_str.columns[0]
        str_start = cis_strs["start"].values[j]
        locus_str['x1'] = locus_str[test_str].apply(lambda x: x.split(',')[0] )
        locus_str['x2'] = locus_str[test_str].apply(lambda x: x.split(',')[1] )
        samples_to_keep = [str_samples[k] for k in range(len(str_samples)) if (str(locus_str['x1'].values[k]) != "NA")and(str(locus_str['x2'].values[k]) != "NA")]   
        locus_str = locus_str.loc[samples_to_keep,:]        
        locus_y = y.loc[samples_to_keep,:]            
        Locus_data = locus_str.join(locus_y)
        locus_str = locus_str.join(locus_y)[['expr','x1','x2']]
        #print(locus_str.head(3), '\n...\n',Locus_detail.head(3))
        # Run regression

        data = Locus_data[['expr','x1','x2']]
        quad_model, alpha, alpha_se, alpha_pval, beta, beta_se, beta_pval=QuadraticRegression(data,norm=NORM, minsamples=120)
        data = Locus_data[['x1','x2']]
        lin_model, slope, err, pval = LinearRegression(data, locus_y["expr"].values, norm=NORM, minsamples=MINSAMPLES, alleles=ALLELE)
        quad_rsq, lin_rsq, delta_aic, anova_pval = Modelcompare(lin_model , quad_model)
        print("\t".join(map(str, [CHROM, gene, test_str, str_start,alpha, alpha_se, alpha_pval, beta, beta_se, beta_pval, slope, err, pval,quad_rsq, lin_rsq, delta_aic, anova_pval] ) )+"\n")                
               
    #break
#Locus_detail 

 Getting data for ENSG00000171103.6
ENSG00000171103.6 	 29072687 	 29093167 	 2 STRs tested 



/usr/lib64/python3.4/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/lib64/python3.4/site-packages/pandas/core/indexing.py:621: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value
/usr/lib/python3.4/site-packages/ipykernel/__main__.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pand

chr2	ENSG00000171103.6	STR_29038150	29038150	-0.010851676566984284	0.011284104308154545	0.337271898430781	0.00022074785619383532	0.0011276507494111816	0.8449802664061722	-0.12759267136920582	0.039004515779694875	0.0012422600638011306	0.04618424993982573	0.01811105692312842	502.40275283920045	3.3268933434158005e-109

chr2	ENSG00000171103.6	STR_29069589	29069589	0.0073341646778053715	0.005821412881336053	0.20913163788042413	-0.00027490467339301265	0.0007196985832219563	0.702872126839113	0.13932019202486137	0.03802314188481787	0.00031432123078303765	0.06008959684531301	0.019910506032253417	808.0295831526171	1.503411350724277e-174

 Getting data for ENSG00000163811.7
ENSG00000163811.7 	 29117509 	 29171088 	 4 STRs tested 

chr2	ENSG00000163811.7	STR_29038150	29038150	0.003192262842710153	0.004269104866608933	0.45540753752970065	-0.0003435245992730123	0.00042662307708974024	0.421567224342143	-0.015718247857932424	0.021415215443870445	0.46374151076268766	0.0024317208975562776	0.002953152254

/usr/lib64/python3.4/site-packages/statsmodels/regression/linear_model.py:1386: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


chr2	ENSG00000119820.6	STR_32505190	32505190	3.729655473350135e-17	8.395222547838656e-17	0.6573061711913004	-9.540979117872439e-18	1.4061487138819093e-17	0.49817951146845196	0.020582247522028183	0.02064665535264998	0.3199431578018023	0.004600922847340261	-inf	13741.935615791375	0.0

chr2	ENSG00000119820.6	STR_32558947	32558947	-2.198762005800603e-16	6.842044103691205e-16	0.7482015712215596	-2.4543626679640607e-17	9.230344480250396e-18	0.008334912262140132	-0.016612191489495335	0.018617680965891347	0.37308060539321886	0.00310037450458045	-inf	15258.155721709074	0.0

chr2	ENSG00000119820.6	STR_32564667	32564667	-5.828670879282072e-16	1.346669663931154e-16	2.278274431357006e-05	2.0816681711721685e-17	2.5824307811663226e-17	0.4210582553698813	0.03771257839295871	0.019947017947847773	0.05996933327828108	0.015776310525757453	-7.799107142857142	14803.336921897457	0.0

chr2	ENSG00000119820.6	STR_32583093	32583093	-2.0296264668928643e-16	1.1151560192885025e-16	0.07016343336215168	6.917209860457

AttributeError: 'NoneType' object has no attribute 'rsquared'

In [44]:
data = Locus_data[['expr','x1','x2']]
data
QuadraticRegression(data,norm=NORM, minsamples=120)


/usr/lib64/python3.4/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


(None, None, None, None, None, None, None)

In [40]:
lin_model
quad_model